### The composition over inheritance principle 

In [1]:
# imports 
import sys
import syslog

In [9]:
# initial class
class Logger(object):
    def __init__(self, file):
        self.file = file
    
    def log(self, message):
        self.file.write(message + '\n')
        self.file.flush()

# now, we need two more classes that send message to elsewhere

class SocketLogger(Logger):
    def __init__(self, sock):
        self.sock = sock
    
    def log(self, message):
        self.sock.sendall(message + '\n').encode('ascii')

class SyslogLogger(Logger):
    def __init__(self, priority):
        self.priority = priority
    
    def log(self, message):
        syslog.syslog(self.priority, message)

# Now if we want to add filters, we can create a filtered logger creating a new subclass

class FilteredLogger(Logger):
    def __init__(self, pattern, file):
        self.pattern = pattern
        super().__init__(file)
    
    def log(self, message):
        if self.pattern in message:
            super().log(message)

# call filtered logger
f = FilteredLogger('Error', sys.stdout)
f.log('Ignored: this is not important')
f.log('Error: but you want to see this')

Error: but you want to see this
